# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Calculation started (calculation_id=d0c964b0-c23e-766d-7cfa-4debff0b0313) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import DecimalType

Calculation started (calculation_id=f8c964b0-c392-1994-9892-aeca7f77e3a3) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=d2c964b0-c4c0-68bb-202e-accab0d1ae93) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'rma86'

Calculation started (calculation_id=62c964b0-c632-77ee-a3f6-8cda56f9a86d) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    return trips.filter(trips["trip_distance"] >= 2)

Calculation started (calculation_id=f8c964b0-c747-e7ce-dcf9-aad73d505b67) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [15]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    mtrips = (
        trips
        .join(lookup.filter(lookup["Borough"] == "Manhattan"), trips.DOLocationID == lookup.LocationID, how="inner")
        .withColumn("passenger_count", col("passenger_count").cast("int"))
        .groupBy("DOLocationID")
        .agg(sum("passenger_count").alias("pcount"))
        .orderBy(col("pcount"), ascending=False)
        .limit(20)
    )
    return mtrips
                

Calculation started (calculation_id=e0c964b3-6fe7-8ed8-4f38-9c4a23257f59) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [16]:
mtrips = manhattan_trips(trips, lookup)
mtrips.show()

Calculation started (calculation_id=bec964b3-87e0-9d8d-5af0-4c4f3fef0025) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+-------+
|DOLocationID| pcount|
+------------+-------+
|         236|3528776|
|         230|3162933|
|         161|3116127|
|         162|2714476|
|          48|2678151|
|         239|2595985|
|         170|2529140|
|         231|2502147|
|          79|2495986|
|         238|2423210|
|         142|2386885|
|         237|2330464|
|         263|2285207|
|         141|2190292|
|          68|2182082|
|         186|2104706|
|          13|2064024|
|         163|2041180|
|         246|2005676|
|         140|2000394|
+------------+-------+



## 3.4 [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [23]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    total_agg = trips.groupBy('PULocationID').agg(
        avg('total_amount').alias('avg_total_amount'),
        count('*').alias('total_trips')
    )
    
    trips_to_top = trips.join(mtrips, on='DOLocationID', how='inner')
    
    trips_to_top_agg = trips_to_top.groupBy('PULocationID').agg(
        count('*').alias('trips_to_top_dests')
    )
    
    joined = total_agg.join(trips_to_top_agg, on='PULocationID', how='inner')
    
    wp = joined.withColumn(
        'proportion',
        col('trips_to_top_dests') / col('total_trips')
    ).withColumn(
        'weighted_profit',
        (col('proportion') * col('avg_total_amount')).cast(DecimalType(12, 8))
    ).select(
        'PULocationID',
        'weighted_profit'
    )
    
    return wp

Calculation started (calculation_id=30c964be-9460-9976-6f4f-3dd2732b67e4) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.5 [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [19]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    final = (
        wp.join(lookup, wp.PULocationID == lookup.LocationID, how='inner')
            .select("Zone", "Borough", "weighted_profit")
            .sort(desc("weighted_profit"))
            .limit(20)
            )
    
    return final

Calculation started (calculation_id=96c964ba-793c-da0d-a4ea-bf1e3f292258) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [24]:
bucket = 's3://cse6242-rma86/output-large6'
main('large', bucket)

Calculation started (calculation_id=62c964be-a7c9-2678-f580-e0a385529aa2) in (session=a6c9648c-6a4a-0469-997f-e3dd58517e53). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation 62c964be-a7c9-2678-f580-e0a385529aa2 failed
rma86
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|        Baisley Park|       Queens|    29.36045578|
|Flushing Meadows-...|       Queens|    27.30484573|
|       South Jamaica|       Queens|    26.29491624|
|     Randalls Island|    Manhattan|    24.15098994|
|        Astoria Park|       Queens|    21.70641711|
|Briarwood/Jamaica...|       Queens|    19.94506463|
|Springfield Garde...|       Queens|    19.46830929|
|             Jamaica|       Queens|    19.28394300|
|              Corona|       Queens|    18.22876925|
|   LaGuardia Airport|       Queens|    18.18133881|
|         Jamaica Bay|       Queens|    17.10052945|
|             Maspeth|       Queens|    17.00545064|
|Eltingville/Annad...|Staten Island|    16.83776476|
|         JFK Airport|       Queens|    16.77772535|
|        Batte